## 3. Use-cases and preparations for the petabyte scale

**Requirement:** First run #1

In [1]:
#From #2
import xarray as xr
port=9000
hostname=!echo $HOSTNAME
hosturl="https://"+hostname[0]+":"+str(port)
dsname="example"
zarr_url='/'.join([hosturl,"datasets",dsname,"zarr"])
storage_options=dict(verify_ssl=False)
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
)

### Server-side processing

In case bandwidth is the bottle-neck for users, we can help them with providing some server-side computing resources to e.g. further lossy compress our data. In the following, we firstly store the first 5 years on disk as provided. Secondly, we restart the app with enabled lossy compression and do the same. Afterwards, we compare speed and sizes.

In [23]:
import numpy as np
def prepare_for_storage(ds_to_store):
    bnds=["lat_bnds","lon_bnds"]
    for a in bnds:
        ds_to_store[a]=ds_to_store[a].isel(time=0).squeeze()
    ds_to_store=ds_to_store.set_coords(bnds)
    ds_to_store["time_bnds"].load()

    return ds_to_store

def store_

In [24]:
ds5=ds.where(ds.time.dt.year.isin(range(2015,2020)),drop=True)
ds5=prepare_for_storage(ds5)

In [25]:
!rm compressed.nc

In [28]:
import time
s=time.time()
ds5.to_netcdf(
    "compressed.nc",
    unlimited_dims=["time"],
    encoding=dict(tas=dict(compression="zlib"))
)
e=time.time()
print(e-s, " seconds")
!ls -lrths compressed.nc

2.7509965896606445  seconds
512 -rw-r--r-- 1 k204210 bm0021 6.6M Jan  7 10:51 compressed.nc


In [29]:
import xarray as xr
xr.open_dataset("compressed.nc").load()

<xarray.Dataset> Size: 18MB
Dimensions:    (lat: 192, bnds: 2, lon: 384, time: 60)
Coordinates:
    lat_bnds   (lat, bnds) float64 3kB -89.75 -88.82 -88.82 ... 88.82 89.75
    lon_bnds   (lon, bnds) float64 6kB -0.4688 0.4688 0.4688 ... 358.6 359.5
    height     float64 8B 2.0
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 480B 2015-01-16T12:00:00 ... 2019-12-16T...
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 18MB 243.3 243.3 243.3 ... 251.1 251.1
    time_bnds  (time, bnds) datetime64[ns] 960B 2015-01-01 ... 2020-01-01
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

In [41]:
!ps -ef | grep cloudify

k204210  1101280 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-99eb3193-dfc0-45c3-9953-a1f29fb0888d.json
k204210  1101285 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-0ac5cfbe-7cc6-4136-870a-cbac9aa18a7b.json
k204210  1102668 1100112  0 09:10 ?        00:00:21 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-f59a40b2-6d4d-41dd-b768-7ee9040e362a.json
k204210  1109706 1100112  0 10:42 ?        00:00:01 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-edc24184-27e4-4966-beb3-6f926e2d6cf0.json
k204210  1109938 1100112  2 10:43 ?        00:00:22 /wor

In [42]:
!kill 1111019

In [34]:
import os
os.environ["L_LOSSY"]="1"

In [35]:
%%bash --bg

source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/tas/gn/v20190710/*.nc

In [36]:
dslossy=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
)

In [37]:
ds5lossy=dslossy.where(ds.time.dt.year.isin(range(2015,2020)),drop=True)
ds5lossy=prepare_for_storage(ds5lossy)

In [38]:
!rm lossy_compressed.nc

rm: cannot remove 'lossy_compressed.nc': No such file or directory


In [39]:
s=time.time()
ds5lossy.to_netcdf(
    "lossy_compressed.nc",
    unlimited_dims=["time"],
    encoding=dict(tas=dict(compression="zlib"))
)
e=time.time()
print(e-s, " seconds")
!ls -lrths lossy_compressed.nc

2.4706828594207764  seconds
512 -rw-r--r-- 1 k204210 bm0021 3.5M Jan  7 10:53 lossy_compressed.nc


In [40]:
import xarray as xr
xr.open_dataset("lossy_compressed.nc").load()

<xarray.Dataset> Size: 18MB
Dimensions:    (lat: 192, bnds: 2, lon: 384, time: 60)
Coordinates:
    lat_bnds   (lat, bnds) float64 3kB -89.75 -88.82 -88.82 ... 88.82 89.75
    lon_bnds   (lon, bnds) float64 6kB -0.4688 0.4688 0.4688 ... 358.6 359.5
    height     float64 8B 2.0
  * lat        (lat) float64 2kB -89.28 -88.36 -87.42 ... 87.42 88.36 89.28
  * lon        (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * time       (time) datetime64[ns] 480B 2015-01-16T12:00:00 ... 2019-12-16T...
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 18MB 243.3 243.3 243.3 ... 251.1 251.1
    time_bnds  (time, bnds) datetime64[ns] 960B 2015-01-01 ... 2020-01-01
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP AerChemMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d6f0f6b2-b4c2-460b-bc5e-f1c72f932399

We can see that

- uncompressed data is 16MB, compressed is 6MB, lossy compressed is 4MB.
- the accuray is on the second decimal: 243.30435 becomes 243.3125

### Interest in a region of global ESM output stored in records

A "record" refers to the GRIB-record where the entire global field is stored as one binary chunk. A chunk is the most fine-grained level of data access possible.

Especially for high-resolution data, clients may not want to retrieve the full entity of a storage chunk, e.g. an entire global field or an entire month on hourly data, because this results in large data volumes. In such cases, where we know the data access *pattern*, we can adapt the chunk setting of our provided zarr data set to the use case. The cloudify service allows to provide smaller chunk sizes to clients than the orignal storage chunk sizes by providing some server-side computing resources for rechunking and acting as an intermediate layer between data and client. Although the splitting of storage chunks reduces performance on the server, the benefit of bandwidth reduction can be more important.

**Chunk setting**

Best practice is to set the chunks when opening the data. For our test zarr dataset, we split the chunks in both spatial dimensions into half of the original. In each dimension, a chunk needs to cover at least 1 increment i.e. has to be an integer > 0. 

With the `chunks` keyword in the `open_mfdataset` command, it is controlled how the dataset is chunked. These chunks are mapped to *zarr* chunks of the *zarr* API. 

In our example script, we control that chunk setting through specific environment variables:

In [ ]:
new_lon_chunk_size=int(len(ds["lon"])/2)
new_lat_chunk_size=int(len(ds["lat"])/2)
print(new_lon_chunk_size, new_lat_chunk_size)

We set these as environment variables and restart our app which finds these.

In [ ]:
os.environ["XPUBLISH_LON_CHUNK_SIZE"]=str(new_lon_chunk_size)
os.environ["XPUBLISH_LAT_CHUNK_SIZE"]=str(new_lat_chunk_size)
#to get back storage chunks:
#del os.environ["XPUBLISH_LON_CHUNK_SIZE"]
#del os.environ["XPUBLISH_LAT_CHUNK_SIZE"]

#kill the existing process

In [ ]:
!ps -ef | grep cloudify

In [ ]:
!kill 814927

In [ ]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/tas/gn/v20190710/*.nc

In [ ]:
ds2=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options,
    chunks={}
)
ds2

### Towards virtually, highly aggregated datasets that include multiple variables of the same kind

Based on experience, large aggregations are beneficial for data analysis as users can skip finding and merging data sources. E.g. to train an AI model, it helps to simplify the random access within a complete experiment. With cloudify, we can realize a virtual, highly aggregated, dataset that covers the full time series of all variables that share dimensions.

In our example, we can try to concat and merge *all* monthly atmospheric variables more variables. For that, we use a wildcard for variables in the DRS path.

In [ ]:
!ps -ef | grep cloudify

In [ ]:
#to get back storage chunks:
del os.environ["XPUBLISH_LON_CHUNK_SIZE"]
del os.environ["XPUBLISH_LAT_CHUNK_SIZE"]

In [ ]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_example.py \
    example \
    /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/*/gn/v20190710/*.nc

In [ ]:
ds=xr.open_zarr(
    zarr_url,
    consolidated=True,
    storage_options=storage_options
)

In [ ]:
ds

In [ ]:
ds.nbytes/1024**3

In [ ]:
ds.isel(time=0,plev=0,lev=0).drop(["height","ap","ap_bnds","b","b_bnds","lev_bnds"]).load()

We see that

- we host O(100GB) of data "just like that" because of lazy data access.
- the memory usage increases with the number of dask chunks to keep in memory. This is the bottle neck for our data server.
- the more data we aggregate into one dataset, the less endpoints we provide. Thus, the catalog of endpoints becomes small and we use the zarr datasets as the *real* catalogs.
    - this simplifies random access which can be crucial for training of AI models

In the next episode, we use *kerchunks* to avoid repeatedly merging these files when opening and instead use a once-*prepared*  virtual dataset representation of this aggrgeation.